In [50]:
#import Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [51]:
# The code was removed by Watson Studio for sharing.

,id,zip,province,district,lat,lng
0,1,10200,กรุงเทพมหานคร,พระนคร,13.756024,100.498679
1,2,10300,กรุงเทพมหานคร,ดุสิต,13.772694,100.509926
2,3,10530,กรุงเทพมหานคร,หนองจอก,13.855988,100.861969
3,4,10500,กรุงเทพมหานคร,บางรัก,13.726240,100.526799
4,5,10220,กรุงเทพมหานคร,บางเขน,13.864387,100.614643


In [52]:
#Processing Data
geo_data.rename(columns = {'zip': 'postcode'}, inplace = True)
geo_data['coordinates'] = list(zip(geo_data['lat'], geo_data['lng']))
geo_data['coordinates'] = geo_data['coordinates'].apply(Point)
geo_data.head()

,id,postcode,province,district,lat,lng,coordinates
0,1,10200,กรุงเทพมหานคร,พระนคร,13.756024,100.498679,POINT (13.7560243 100.4986793)
1,2,10300,กรุงเทพมหานคร,ดุสิต,13.772694,100.509926,POINT (13.7726943 100.5099262)
2,3,10530,กรุงเทพมหานคร,หนองจอก,13.855988,100.861969,POINT (13.8559883 100.8619689)
3,4,10500,กรุงเทพมหานคร,บางรัก,13.726240,100.526799,POINT (13.7262395 100.5267991)
4,5,10220,กรุงเทพมหานคร,บางเขน,13.864387,100.614643,POINT (13.864387 100.6146434)


In [53]:
# import map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON 
import geopy

# use geopy to obtain latitude/longitude of Toronto.
# coordinates will be used for map visualization

address = 'Bangkok, Thailand'

geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangkok {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangkok 13.7538929, 100.8160803.


In [54]:
import folium

# create map of Toronto using latitude and longitude values
map_bangkok = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, neighbourhood in zip(geo_data['lat'], geo_data['lng'], geo_data['district']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangkok)  
    
map_bangkok

In [55]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'JRG2WVLJKBPVXW3TB2L3NGNSGPNFBBGRDERMQU4OSN3ZMDQF' # your Foursquare ID
CLIENT_SECRET = 'H3DKQHPVZJJIPF4MKAK5AUJ3PNKAEVQ3OWPSY3ALCCUQWE23' # your Foursquare Secret
VERSION = '20161225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JRG2WVLJKBPVXW3TB2L3NGNSGPNFBBGRDERMQU4OSN3ZMDQF
CLIENT_SECRET:H3DKQHPVZJJIPF4MKAK5AUJ3PNKAEVQ3OWPSY3ALCCUQWE23


In [56]:
# define limit = 5 (limit to 5 venues only) & radius = 500 (meters)
LIMIT = 5
radius = 500

location_list = [] # initiate a list to store data from Foursquare API requests

for neighbourhood, latitude, longitude in zip(geo_data.district, geo_data.lat, geo_data.lng):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json()
    #print(data)
    # use len() to check if any data within "items" (len = 0 if nothing)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue # skip the row if nothing is found
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # extract info within 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        location_list.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])

In [66]:
# create data frame based on 'Location_list'

temp = pd.DataFrame(x for row in location_list for x in row)
temp.columns = ['district','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','category']
temp.head()

print("{} nearby locations downloaded for {} neighbourhood.".format(len(temp.Venue), len(geo_data.district)))

49 nearby locations downloaded for 50 neighbourhood.


In [68]:
cat = pd.get_dummies(temp.category) # one hot encoding with get_dummies()
df_01 = pd.concat([temp[['district']], cat], axis=1) # combine neighbourhood & category tables
df_01.head()

,district,Asian Restaurant,Bar,Beer Bar,Buddhist Temple,Buffet,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,...,Hotpot Restaurant,Ice Cream Shop,Japanese Restaurant,Noodle House,Palace,Park,Steakhouse,Thai Restaurant,Train Station,Wine Bar
0,พระนคร,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ดุสิต,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,หนองจอก,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,บางรัก,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,บางเขน,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# Cluster Analysis

In [58]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [69]:
df_02 = df_01.drop('district', axis=1)

n_group = 6 # we will group neighbourhoods into 6 clusters

# run k-means clustering
kmeans = KMeans(n_clusters=n_group, random_state=0).fit(df_02)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
df_01.insert(1, 'label', kmeans.labels_)

df_01.head() 


,district,label,Asian Restaurant,Bar,Beer Bar,Buddhist Temple,Buffet,Chinese Restaurant,Clothing Store,Coffee Shop,...,Hotpot Restaurant,Ice Cream Shop,Japanese Restaurant,Noodle House,Palace,Park,Steakhouse,Thai Restaurant,Train Station,Wine Bar
0,พระนคร,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ดุสิต,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,หนองจอก,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,บางรัก,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,บางเขน,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [70]:
# merge 'geo_toronto' & df_01
bkk_merged = pd.merge(geo_data, df_01, on='district', how='right')
bkk_merged.head()

,id,postcode,province,district,lat,lng,coordinates,label,Asian Restaurant,Bar,...,Hotpot Restaurant,Ice Cream Shop,Japanese Restaurant,Noodle House,Palace,Park,Steakhouse,Thai Restaurant,Train Station,Wine Bar
0,1,10200,กรุงเทพมหานคร,พระนคร,13.756024,100.498679,POINT (13.7560243 100.4986793),0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,10300,กรุงเทพมหานคร,ดุสิต,13.772694,100.509926,POINT (13.7726943 100.5099262),0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,10530,กรุงเทพมหานคร,หนองจอก,13.855988,100.861969,POINT (13.8559883 100.8619689),0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,10500,กรุงเทพมหานคร,บางรัก,13.726240,100.526799,POINT (13.7262395 100.5267991),0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,10220,กรุงเทพมหานคร,บางเขน,13.864387,100.614643,POINT (13.864387 100.6146434),4,0,0,...,0,0,0,1,0,0,0,0,0,0


In [71]:
from matplotlib import cm, colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_group)
ys = [i + x + (i*x)**2 for i in range(n_group)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bkk_merged['lat'], bkk_merged['lng'], bkk_merged['district'], bkk_merged['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters